# Session 17: Concrete Formulation I (with Solutions)

## Steps for solving a practical optimization problem

**1. English description** (for conceptual understanding).

**2. Concrete formulation** (using concrete numbers in a toy example).

**3. Abstract formulation** (using data variables and summation notation).

**4. Reusable software** (that works with any input file of a certain format). 

## Q1. Supply Chain Planning

The following table provides the shipping cost for a certain item, from 3 of Amazon’s fulfillment centers (FC) to 4 regions (A, B, C and D). 

| Region \ FC  | 1 | 2 | 3 |
|--|--|--|--|
| A. Kings County, NY | 20 | 8 | 25 | 
| B. Los Angeles County, CA | 18 | 23 | 8 | 
| C. King County, WA | 21 | 24 | 8 | 
| D. Harris County, TX | 8 | 8 | 19 |

The following table summarizes the weekly demand for the item from each region.

 | Region A | Region B | Region C | Region D|
|--|--|--|--|
|30 | 50 | 10 | 20 |

Suppose that each FC is able to ship up to 40 units to any region each week. Formulate a linear program to determine the minimum transportation cost needed to satisfy all demand while respecting FC capacities, as well as the optimal shipment plan. Moreover, interpret the meaning of the shadow price of each constraint and predict its sign (positive or negative).

### Step 1. Identify the decision, objective, and constraints in English

**Decision:** How many units to transport from each FC to each region.

**Objective:** Minimize transportation cost.

**Constraints:**

- Not transporting more out of a FC than its capacity.
- Transporting enough to each region to meet its demand. 

### Step 2. Formulate the optimization as linear expressions of decision variables

**Decision variables:** Let $x_{ij}$ denote the amount to transport from FC $i$ to region $j$. (Continuous)


**Objective and Constraints:** 

$$
\begin{aligned}
\text{Minimize: } && 20x_{1A}+18x_{1B}+21x_{1C}+8x_{1D} +\\ && 8x_{2A}+23x_{2B}+24x_{2C}+8x_{2D}+\\
&& 25x_{3A}+8x_{3B}+8x_{3C}+19x_{3D} \\
\text{Subject to:}\\
\text{(Capacity 1)} && x_{1A}+x_{1B}+x_{1C}+x_{1D} & \le 40 \\
\text{(Capacity 2)} && x_{2A}+x_{2B}+x_{2C}+x_{2D} & \le 40 \\
\text{(Capacity 3)} && x_{3A}+x_{3B}+x_{3C}+x_{3D} & \le 40 \\
\text{(Demand A)} && x_{1A}+x_{2A}+x_{3A} & \ge 30 \\
\text{(Demand B)} && x_{1B}+x_{2B}+x_{3B} & \ge 50 \\
\text{(Demand C)} && x_{1C}+x_{2C}+x_{3C} & \ge 10 \\
\text{(Demand D)} && x_{1D}+x_{2D}+x_{3D} & \ge 20 \\
\text{(Non-negativity}) && x_{ij} & \ge 0 \quad \text{for all $i$ and $j$.}
\end{aligned}$$

### Interpretation of Shadow Prices

The shadow price of each capacity constraint is how would the optimal objective value change if the capacity of each fulfillment center is increased by 1. We would expect possible additional cost savings from increasing capacity, so the shadow price should be negative (or zero).

The shadow price of each demand constraint is how would the optimal cost change if each region increases their requirement by 1 unit. This would impose additional stress on the supply chain and we would expect the total transportation cost to increase as a result. Hence, the shadow price should be positive.


## Q2. Gurobi Practice

Solve Q1 using Gurobi and find the shadow prices of and valid ranges of all the capacity constraints. For your reference, here is the sample code from session 13. The code solves the LP

$$\begin{aligned} & \text{Maximize:} & 100X+300Y \\
& \text{subject to:} \\
& \text{(Alice)} & X+2Y & \le 6 \\
& \text{(Bob)} & 3Y & \le 6 \\
& \text{(Non-negativity)} & X, Y & \ge 0
\end{aligned}$$

In [4]:
from gurobipy import Model, GRB
mod=Model()                   
X=mod.addVar(lb=0)            # lb: lower bound of decision variable
Y=mod.addVar(lb=0)
mod.setObjective(100*X+300*Y,sense=GRB.MAXIMIZE)
alice=mod.addConstr(X+2*Y<=6)   # constraint object is named only to find shadow prices later
mod.addConstr(3*Y<=6)
mod.setParam('OutputFlag',False) # Hides diagnostic ouputs during optimization (not necessary)
mod.optimize()
print('Optimal objective: ',mod.objVal)
print(f'Optimal plan: X={X.x} Y={Y.x}')  # Always .x to find the optimal value of a decision variable
print(f'Shadow price of Alice constraint: {alice.pi}')
print(f'(Valid range for RHS of Alice constraint: {alice.sarhslow} and {alice.sarhsup})')

Optimal objective:  800.0
Optimal plan: X=2.0 Y=2.0
Shadow price of Alice constraint: 100.0
(Valid range for RHS of Alice constraint: 4.0 and inf)


In [2]:
from gurobipy import Model, GRB
mod=Model()
x1A=mod.addVar(lb=0)
x1B=mod.addVar(lb=0)
x1C=mod.addVar(lb=0)
x1D=mod.addVar(lb=0)
x2A=mod.addVar(lb=0)
x2B=mod.addVar(lb=0)
x2C=mod.addVar(lb=0)
x2D=mod.addVar(lb=0)
x3A=mod.addVar(lb=0)
x3B=mod.addVar(lb=0)
x3C=mod.addVar(lb=0)
x3D=mod.addVar(lb=0)
mod.setObjective(20*x1A+18*x1B+21*x1C+8*x1D+8*x2A+23*x2B+24*x2C+8*x2D+25*x3A+8*x3B+8*x3C+19*x3D)
cap1=mod.addConstr(x1A+x1B+x1C+x1D<=40)
cap2=mod.addConstr(x2A+x2B+x2C+x2D<=40)
cap3=mod.addConstr(x3A+x3B+x3C+x3D<=40)
mod.addConstr(x1A+x2A+x3A>=30)
mod.addConstr(x1B+x2B+x3B>=50)
mod.addConstr(x1C+x2C+x3C>=10)
mod.addConstr(x1D+x2D+x3D>=20)
mod.setParam('OutputFlag',False)
mod.optimize()
print('Minimum transportation cost:',mod.objval)
print('Optimal shipment plan:')
print(f'\tx1A={x1A.x} x1B={x1B.x} x1C={x1C.x} x1D={x1D.x}')
print(f'\tx2A={x2A.x} x2B={x2B.x} x2C={x2C.x} x2D={x2D.x}')
print(f'\tx3A={x3A.x} x3B={x3B.x} x3C={x3C.x} x3D={x3D.x}')
print('Capacity 1 constraint')
print(f'\tShadow Price={cap1.pi}  Valid when RHS is in [{cap1.sarhslow},{cap1.sarhsup}]')
print('Capacity 2 constraint')
print(f'\tShadow Price={cap2.pi}  Valid when RHS is in [{cap2.sarhslow},{cap2.sarhsup}]')
print('Capacity 3 constraint')
print(f'\tShadow Price={cap3.pi}  Valid when RHS is in [{cap3.sarhslow},{cap3.sarhsup}]')

Minimum transportation cost: 1080.0
Optimal shipment plan:
	x1A=0.0 x1B=20.0 x1C=0.0 x1D=20.0
	x2A=30.0 x2B=0.0 x2C=0.0 x2D=0.0
	x3A=0.0 x3B=30.0 x3C=10.0 x3D=0.0
Capacity 1 constraint
	Shadow Price=0.0  Valid when RHS is in [40.0,inf]
Capacity 2 constraint
	Shadow Price=0.0  Valid when RHS is in [30.0,inf]
Capacity 3 constraint
	Shadow Price=-10.0  Valid when RHS is in [40.0,60.0]


## (Optional) Q3. Optimal Assignment

There are four consultants, Alice, Bob, Charles, and Daphne, who must be assigned to one of two projects. Each consultant can only be assigned to one project, and each project requires two consultants. As a manager, you evaluated the relative fitness of the four consultants for each project on a scale of 1 to 5, with 5 being the best fit and 1 being the worst.

| ` `| Project 1 | Project 2 |
|--|--|--|
|Alice | 5 | 2 |
|Bob | 3 | 2 |
|Charles | 4 | 5 |
|Daphne | 3 | 1 |
 
Furthermore, Alice, Bob and Daphne are senior consultants and each project requires at least one senior on the team. 

**a)** Formulate an optimization problem to maximize the total fitness of the consultants to their assigned project, subject to all the business constraints.

**b)** Numerically solve the optimization using Gurobi. (Only difference with binary or integer variables in Gurobi is that you need to do `mod.addVar(vtype=GRB.BINARY)` `mod.addVar(vtype=GRB.INTEGER)` or `mod.addVar(lb=0,vtype=GRB.INTEGER)`)

### Step 1. Identify the decision, objective, and constraints in English

**Decision:** Which project to assign to each consultant.

**Objective:** Maximize total fitness.

**Constraints:**

- Each consultant can be assigned to at most one project.
- Each project requires at least two consultants assigned.
- Each project requires at least one senior consultant assigned.

### Step 2. Formulate the optimization as linear expressions of decision variables

**Decision variables:** Let $x_{ij}$ denote whether to assign consultant $i$ to project $j$. (Binary)

**Objective and Constraints:** 
$$
\begin{aligned}
\text{Maximize: } && 5x_{A1}+2x_{A2}+3x_{B1}+2x_{B2} +\\ 
&& 4x_{C1}+5x_{C2}+3x_{D1}+x_{D2} \\
\text{Subject to:}\\
\text{(Alice)} && x_{A1}+x_{A2} & \le 1 \\
\text{(Bob)} && x_{B1}+x_{B2} & \le 1 \\
\text{(Charles)} && x_{C1}+x_{C2} & \le 1 \\
\text{(Daphne)} && x_{D1}+x_{D2} & \le 1 \\
\text{(Project 1 Total)} && x_{A1}+x_{B1}+x_{C1}+x_{D1} & \ge 2 \\
\text{(Project 2 Total)} && x_{A2}+x_{B2}+x_{C2}+x_{D2} & \ge 2 \\
\text{(Project 1 Senior)} && x_{A1}+x_{B1}+x_{D1} & \ge 1 \\
\text{(Project 2 Senior)} && x_{A2}+x_{B2}+x_{D2} & \ge 1 \\
\end{aligned}$$

In [3]:
# Numerical solution
from gurobipy import Model, GRB
mod=Model()
A1=mod.addVar(vtype=GRB.BINARY)
A2=mod.addVar(vtype=GRB.BINARY)
B1=mod.addVar(vtype=GRB.BINARY)
B2=mod.addVar(vtype=GRB.BINARY)
C1=mod.addVar(vtype=GRB.BINARY)
C2=mod.addVar(vtype=GRB.BINARY)
D1=mod.addVar(vtype=GRB.BINARY)
D2=mod.addVar(vtype=GRB.BINARY)
mod.setObjective(5*A1+2*A2+3*B1+2*B2+4*C1+5*C2+3*D1+D2, sense=GRB.MAXIMIZE)
mod.addConstr(A1+A2<=1)
mod.addConstr(B1+B2<=1)
mod.addConstr(C1+C2<=1)
mod.addConstr(D1+D2<=1)
mod.addConstr(A1+B1+C1+D1>=2)
mod.addConstr(A2+B2+C2+D2>=2)
mod.addConstr(A1+B1+D1>=1)
mod.addConstr(A2+B2+D2>=1)
mod.setParam('OutputFlag',False)
mod.optimize()
print('Maximum total fitness:',mod.objval)
print('Optimal Assignment:')
print(f'\tAlice assigned to Project {1 if A1.x else 2}.')
print(f'\tBob assigned to Project {1 if B1.x else 2}.')
print(f'\tCharles assigned to Project {1 if C1.x else 2}.')
print(f'\tDaphne assigned to Project {1 if D1.x else 2}.')

Maximum total fitness: 15.0
Optimal Assignment:
	Alice assigned to Project 1.
	Bob assigned to Project 2.
	Charles assigned to Project 2.
	Daphne assigned to Project 1.
